In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

URL_BASE = 'https://www.argenprop.com'


def obtener_enlaces_desde_inicio():
    respuesta = requests.get(URL_BASE)
    sopa = BeautifulSoup(respuesta.content, 'html.parser')

    enlaces = []
    for h3 in sopa.find_all('h3', class_='btn btn-text btn-block'):
        etiqueta_a = h3.find('a')
        if etiqueta_a:
            href = etiqueta_a['href']
            if any(palabra in href for palabra in ["departamento-alquiler", "casa-alquiler", "inmuebles-alquiler"]):
                enlaces.append(href)

    # Excluir los tres primeros enlaces
    return enlaces[3:]


def obtener_todos_enlaces_de_pagina(pagina_url):
    page_number = 1
    all_links = []
    while True:
        if page_number == 1:
            url = pagina_url
        else:
            url = pagina_url + "-pagina-" + str(page_number)
        
        respuesta = requests.get(url)
        
        if respuesta.status_code != 200:
            print(f"Error al obtener la página {page_number}. Código de estado: {respuesta.status_code}")
            break

        sopa = BeautifulSoup(respuesta.content, 'html.parser')
        enlaces_pagina = [enlace['href'] for enlace in sopa.find_all('a', class_='card') if enlace.has_attr('href')]
        
        if not enlaces_pagina:
            break
        
        all_links.extend(enlaces_pagina)
        page_number += 1

    return all_links


def extraer_datos_de_enlace(enlace):
    respuesta = requests.get(URL_BASE + enlace)

    if respuesta.status_code != 200:
        return None

    soup = BeautifulSoup(respuesta.content, 'html.parser')

    try:
        location = soup.find('h3', class_='titlebar__address').text
        price = soup.find('p', class_='titlebar__price').text.strip()
        features_list_element = soup.find('ul', class_='property-main-features')
        features_list = features_list_element.find_all('li') if features_list_element else []
        features = {}
        for feature in features_list:
            key = feature.attrs['title']
            value = feature.find('div', class_='desktop').p.text.strip()
            features[key] = value
        expenses_element = soup.find('p', class_='titlebar__expenses')
        expenses = expenses_element.text.strip() if expenses_element else "No se encontraron expensas"

        data = {
            'URL': URL_BASE + enlace,
            'Ubicación': location,
            'Precio': price,
            **features,
            'Expensas': expenses
        }
        return data
    except Exception as e:
        print(f"Error al extraer datos del enlace: {enlace}. Error: {e}")
        return None


def main():
    enlaces_inicio = obtener_enlaces_desde_inicio()
    todos_enlaces = []

    for enlace in enlaces_inicio:
        todos_enlaces.extend(obtener_todos_enlaces_de_pagina(enlace))

    datos_lista = []
    for enlace in todos_enlaces:
        data = extraer_datos_de_enlace(enlace)
        if data:
            datos_lista.append(data)

    df = pd.DataFrame(datos_lista)

    # Para visualizar el dataframe
    print(df.head())


if __name__ == "__main__":
    main()


KeyboardInterrupt: 

In [28]:

if __name__ == "__main__":
    # Solo como ejemplo, usa una URL específica de tu lista.
    # En un caso real, podrías obtener esta URL llamando primero a la función obtener_enlaces_desde_inicio()
    URL_EJEMPLO = "https://www.argenprop.com/departamento-alquiler-barrio-belgrano"
    
    enlaces = obtener_todos_enlaces_de_pagina(URL_EJEMPLO)
    for enlace in enlaces:
        print(enlace)

['/departamento-en-alquiler-en-belgrano-r--14309236', '/departamento-en-alquiler-en-belgrano-4-ambientes--13601395', '/departamento-en-alquiler-en-belgrano-1-ambiente--12773933', '/departamento-en-alquiler-en-belgrano-c-3-ambientes--14144558', '/departamento-en-alquiler-en-las-canitas-2-ambientes--13659809', '/departamento-en-alquiler-en-belgrano-4-ambientes--13519288', '/departamento-en-alquiler-en-belgrano-4-ambientes--14275631', '/departamento-en-alquiler-en-las-canitas-2-ambientes--14276769', '/departamento-en-alquiler-en-belgrano-3-ambientes--13472228', '/departamento-en-alquiler-en-belgrano-4-ambientes--13264066', '/departamento-en-alquiler-en-belgrano-barrancas-5-ambientes--11341474', '/departamento-en-alquiler-en-belgrano-3-ambientes--13974574', '/departamento-en-alquiler-en-belgrano-3-ambientes--14177904', '/departamento-en-alquiler-en-las-canitas-3-ambientes--14108164', '/departamento-en-alquiler-en-las-canitas-4-ambientes--12675949', '/departamento-en-alquiler-en-belgrano-5-

¡Datos guardados exitosamente en 'propiedades_argenprop.csv'!


In [34]:
def main():
    enlaces_inicio = obtener_enlaces_desde_inicio()
    todos_enlaces = []

    for enlace in enlaces_inicio:
        enlaces_pagina, _ = obtener_todos_enlaces_de_pagina(enlace)  # Usamos '_' para ignorar el segundo valor de la tupla.
        todos_enlaces.extend(enlaces_pagina)

    datos_lista = []
    for enlace in todos_enlaces:
        data = extraer_datos_de_enlace(enlace)
        if data:
            datos_lista.append(data)

    df = pd.DataFrame(datos_lista)

    # Para visualizar el dataframe
    print(df.head())

if __name__ == "__main__":
    main()

KeyboardInterrupt: 